In [1]:
import ipdb
import sys
sys.path.append('../ATAE-LSTM')
import Ipynb_importer
from data.Embedding import Emb
from data.AspClas import AspClas
from models.ATAE_LSTM import ATAE_LSTM
from utils.visualize import Visualizer
from config import opt
from tqdm import tqdm
from random import randint

importing Jupyter notebook from D:\Jupyter\ATAE-LSTM\data\Embedding.ipynb
importing Jupyter notebook from D:\Jupyter\ATAE-LSTM\data\AspClas.ipynb
importing Jupyter notebook from D:\Jupyter\ATAE-LSTM\models\ATAE_LSTM.ipynb
importing Jupyter notebook from D:\Jupyter\ATAE-LSTM\models\BasicModule.ipynb


In [2]:
import torch as t
from torch.utils.data import DataLoader
from torchnet import meter
from torch.autograd import Variable

In [3]:
def val(model, dataloader):
    '''
    计算模型在验证集上的准确率等信息
    '''
    
    confusion_matrix = meter.ConfusionMeter(opt.classes)
    
    with t.no_grad():
        for step, data in enumerate(tqdm(dataloader)):
            sentence, terms, label = data
            score = model(sentence, terms)
            confusion_matrix.add(score.data, label.data.squeeze())
    
    cm_value = confusion_matrix.value()
    accuracy = 100.0 * (cm_value.trace()) / (cm_value.sum())
    return confusion_matrix, accuracy

In [4]:
! python -m visdom.server
vis = Visualizer(opt.env)

Setting up a new session...


In [5]:
%pdb on
# step1 data
train_data = AspClas(opt.train_data_root, test=False, train=True)
test_data = AspClas(opt.test_data_root, test=True, train=False, emb=train_data.emb)
train_dataloader = DataLoader(
    train_data,
    opt.batch_size,
    shuffle=True,
    drop_last = True
)
test_dataloader = DataLoader(
    test_data,
    opt.batch_size,
    shuffle=False,
    drop_last = True
)

Automatic pdb calling has been turned ON


100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:18<00:00, 5302.73it/s]


Embedding : successfully input 100000 pretrained word embeddings while 0 failed


100%|████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 100031.10it/s]


In [6]:
# step2 configure model
model = ATAE_LSTM(emb=train_data.emb)
if opt.use_cuda:
    model = model.cuda()

In [7]:
# step3 criterion and optimizer
criterion = t.nn.CrossEntropyLoss()
lr = opt.lr
optimizer = t.optim.Adam(
    model.parameters(),
    lr = lr,
    weight_decay = opt.weight_decay
)

In [8]:
# step4 meters
loss_meter = meter.AverageValueMeter()
confusion_matrix = meter.ConfusionMeter(opt.classes)
previous_loss = 1e100

In [9]:
# step5 train

# validate and visualize at start
val_cm, val_accuracy = val(model, test_dataloader)
vis.plot('val_accuracy', val_accuracy)
vis.plot('lr', lr)
vis.log("epoch:{epoch},\nlr:{lr},\ntrain_cm:{train_cm},\nval_cm:{val_cm}".format(
    epoch = 0,
    val_cm = str(val_cm.value()),
    train_cm=str(confusion_matrix.value()),
    lr=lr
))

total_step = 0
for epoch in range(opt.max_epoch):
    loss_meter.reset()
    confusion_matrix.reset()
    
    for step, (sentence, terms, label) in enumerate(tqdm(train_dataloader)):
        
        if opt.use_cuda:
            sentence, terms, label = sentence.cuda(), terms.cuda(), label.cuda()
        
        score = model(sentence, terms)
        loss = criterion(score, label.squeeze())
        loss.backward()
        optimizer.step()
        
        # meters update and visualize
        loss_meter.add(loss.data)
        confusion_matrix.add(score.data, label.data.squeeze())
        if total_step%opt.print_freq == 0:
            vis.plot('loss', loss_meter.value()[0])
            vis.log("score:{score},target:{label}".format(
                score = score,
                label = label
            ))
        total_step += 1
        
    model.save()
    
    # validate and visualize
    val_cm, val_accuracy = val(model, test_dataloader)
    
    vis.plot('val_accuracy', val_accuracy)
    vis.plot('lr', lr*1000)
    vis.log("epoch:{epoch},\nlr:{lr},\ntrain_cm:{train_cm},\nval_cm:{val_cm}".format(
        epoch = epoch,
        val_cm = str(val_cm.value()),
        train_cm=str(confusion_matrix.value()),
        lr=lr
    ))
    
    # update learning rate
    if loss_meter.value()[0].item() >= previous_loss:
        lr = lr * opt.lr_decay
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    previous_loss = loss_meter.value()[0]

100%|███████████████████████████████████████████████████████████████████████████████▊| 631/632 [08:35<00:00,  1.35it/s]

RuntimeError: shape '[4, 1, 300]' is invalid for input of size 300

> <string>(81)forward()

ipdb> u
> c:\programdata\anaconda3\lib\site-packages\torch\nn\modules\module.py(532)__call__()
    530             result = self._slow_forward(*input, **kwargs)
    531         else:
--> 532             result = self.forward(*input, **kwargs)
    533         for hook in self._forward_hooks.values():
    534             hook_result = hook(self, input, result)

ipdb> u
> <ipython-input-9-63da5e5698a3>(19)<module>()
     17     for step, (sentence, terms, label) in enumerate(tqdm(train_dataloader)):
     18 
---> 19         score = model(sentence, terms)
     20         loss = criterion(score, label.squeeze())
     21         loss.backward()

ipdb> q
